In [54]:
# import csv

# x, y = [], []
# with open('./datasets/iris/iris.data', 'rb') as csvfile:
#     lines = csv.reader(csvfile)
#     for row in lines:
#         x.append(row[:-2])
#         y.append(row[-1:])
# X, Y = np.array(x), np.array(y)

In [72]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'class2'],
                   ['blue', 'XL', 15.3, 'class1']])
df.columns = ['color', 'size', 'price', 'classlabel']
df



,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [73]:
size_mapping = {'M':1, 'L':2, 'XL':3}
df['size'] = df['size'].map(size_mapping)
df
pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [66]:
# map ordinal label to integers
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'])
# class_mapping = {label : idx for label, idx in enumerate(np.unique(df['classlabel']))}
# print class_mapping
color_le = LabelEncoder()
df['color'] = color_le.fit_transform(df['color'])
df

,color,size,price,classlabel
0,1,1,10.1,class1
1,2,2,13.5,class2
2,0,3,15.3,class1


In [69]:
from sklearn.preprocessing import OneHotEncoder

X = df[['color', 'size', 'price']].values
ohe = OneHotEncoder(categorical_features=[0],sparse=False)
ohe.fit_transform(X)

array([[  0. ,   1. ,   0. ,   1. ,  10.1],
       [  0. ,   0. ,   1. ,   2. ,  13.5],
       [  1. ,   0. ,   0. ,   3. ,  15.3]])

In [85]:
import os
os.system('cls||clear')

df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/wine/wine.data', header=None)
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']
print('Class Labels', np.unique(df_wine['Class label']))
df_wine.head()

('Class Labels', array([1, 2, 3]))


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [88]:
# it always a good idea to retrain the classifier after evaluation on all the training data
from sklearn.cross_validation import train_test_split
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

### Feature Scaling
#except decision tree and random forest, most machine learning algorithm requires feature scaling -> SGD will only optimize weights where larger error occurs.

StandardScaler()

## Overfitting
### the model is too complicated to fit the training data, and could not generalize well on test data set -> high variance model \\
#### regularization
#### collect more training data
#### choose a simpler model with few parameters
#### reduce the dimension of data
 

In [89]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)
lr = LogisticRegression(penalty='l1', C = 0.1)
lr.fit(X_train, y_train)
print('Training accuracy:', lr.score(X_train_std, y_train))
print('Test accuracy:', lr.score(X_test_std, y_test))

('Training accuracy:', 0.86290322580645162)
('Test accuracy:', 0.85185185185185186)


In [93]:
#random forest to decide the predication important of each feature to the model.... 
#however, not to interpret the importance, because some highly correlated features might 
# have one with high importance, the other very small

from sklearn.ensemble import RandomForestClassifier
feat_labels = df_wine.columns[1:]
forest = RandomForestClassifier(n_estimators = 10000, random_state=0, n_jobs=-1)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
for i in range(len(indices)):
    print "%2d) %-*s %f" % (i+1, 30, feat_labels[i], importances[indices[i]])

 1) Alcohol                        0.182508
 2) Malic acid                     0.158574
 3) Ash                            0.150954
 4) Alcalinity of ash              0.131983
 5) Magnesium                      0.106564
 6) Total phenols                  0.078249
 7) Flavanoids                     0.060717
 8) Nonflavanoid phenols           0.032039
 9) Proanthocyanins                0.025385
10) Color intensity                0.022369
11) Hue                            0.022070
12) OD280/OD315 of diluted wines   0.014655
13) Proline                        0.013933
